In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from datetime import datetime, timedelta
import requests
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()
ALPHA_VANTAGE_API_KEY = os.getenv('ALPHA_VANTAGE_API_KEY')

class DeltaFocusedAnalyzer:
    """Streamlined analyzer focused specifically on options delta"""
    
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "https://www.alpha-vantage.co/query"
        self.risk_free_rate = 0.05
        self.rate_limit_delay = 12
    
    def make_api_call(self, params):
        """Make API call with error handling"""
        try:
            import time
            response = requests.get(self.base_url, params=params)
            data = response.json()
            
            if "Error Message" in data:
                print(f"API Error: {data['Error Message']}")
                return None
            if "Note" in data:
                print(f"API Note: {data['Note']}")
                return None
            
            time.sleep(self.rate_limit_delay)
            return data
            
        except Exception as e:
            print(f"Request error: {e}")
            return None
    
    def get_current_price(self, symbol):
        """Get current stock price"""
        params = {
            'function': 'GLOBAL_QUOTE',
            'symbol': symbol,
            'apikey': self.api_key
        }
        
        data = self.make_api_call(params)
        if data and 'Global Quote' in data:
            try:
                return float(data['Global Quote']['05. price'])
            except (KeyError, ValueError):
                pass
        
        # Fallback to daily data
        params = {
            'function': 'TIME_SERIES_DAILY',
            'symbol': symbol,
            'apikey': self.api_key
        }
        
        data = self.make_api_call(params)
        if data and 'Time Series (Daily)' in data:
            try:
                time_series = data['Time Series (Daily)']
                latest_date = max(time_series.keys())
                return float(time_series[latest_date]['4. close'])
            except (KeyError, ValueError):
                pass
        
        return None
    
    def calculate_historical_volatility(self, symbol, days=30):
        """Calculate historical volatility for delta calculations"""
        params = {
            'function': 'TIME_SERIES_DAILY',
            'symbol': symbol,
            'apikey': self.api_key
        }
        
        data = self.make_api_call(params)
        if not data or 'Time Series (Daily)' not in data:
            return 0.25  # Default volatility
        
        try:
            time_series = data['Time Series (Daily)']
            prices = []
            
            for date_str in sorted(time_series.keys(), reverse=True)[:days]:
                prices.append(float(time_series[date_str]['4. close']))
            
            if len(prices) < 2:
                return 0.25
            
            # Calculate daily returns
            returns = []
            for i in range(1, len(prices)):
                returns.append(np.log(prices[i-1] / prices[i]))
            
            daily_vol = np.std(returns)
            annual_vol = daily_vol * np.sqrt(252)
            
            return max(0.05, min(2.0, annual_vol))
            
        except Exception:
            return 0.25
    
    def black_scholes_delta(self, S, K, T, r, sigma, option_type='call'):
        """Calculate Black-Scholes delta - the core function"""
        if T <= 0 or sigma <= 0:
            return 0
        
        try:
            d1 = (np.log(S/K) + (r + sigma**2/2) * T) / (sigma * np.sqrt(T))
            
            if option_type == 'call':
                delta = norm.cdf(d1)
            else:  # put
                delta = norm.cdf(d1) - 1
                
            return delta
        except:
            return 0
    
    def generate_delta_focused_options(self, symbol, current_price, price_range=20):
        """Generate realistic options data focused on delta analysis"""
        options_data = []
        
        # Get volatility for realistic pricing
        volatility = self.calculate_historical_volatility(symbol)
        
        # Generate expiration dates (next 2 weeks)
        today = datetime.now().date()
        expiry_dates = []
        
        # Find next 2 Fridays
        for i in range(1, 21):
            check_date = today + timedelta(days=i)
            if check_date.weekday() == 4 and len(expiry_dates) < 2:
                expiry_dates.append(check_date.strftime('%Y-%m-%d'))
        
        # Generate strikes around current price
        strike_increment = 1.0 if current_price < 100 else 2.5
        min_strike = current_price - price_range
        max_strike = current_price + price_range
        
        strikes = []
        strike = min_strike
        while strike <= max_strike:
            strikes.append(round(strike / strike_increment) * strike_increment)
            strike += strike_increment
        
        # Generate options with focus on delta characteristics
        for expiry in expiry_dates:
            expiry_date = datetime.strptime(expiry, '%Y-%m-%d').date()
            days_to_expiry = (expiry_date - today).days
            time_to_expiry = days_to_expiry / 365.0
            
            if days_to_expiry <= 0:
                continue
            
            for strike in strikes:
                # Calculate deltas
                call_delta = self.black_scholes_delta(
                    current_price, strike, time_to_expiry, 
                    self.risk_free_rate, volatility, 'call'
                )
                
                put_delta = self.black_scholes_delta(
                    current_price, strike, time_to_expiry,
                    self.risk_free_rate, volatility, 'put'
                )
                
                # Only include options with meaningful delta
                if abs(call_delta) > 0.01:
                    options_data.append({
                        'type': 'call',
                        'strike': strike,
                        'expiry': expiry,
                        'days_to_expiry': days_to_expiry,
                        'time_to_expiry': time_to_expiry,
                        'delta': call_delta,
                        'implied_vol': volatility,
                        'moneyness': current_price / strike,
                        'volume': max(1, int(100 * np.exp(-2 * abs(np.log(current_price/strike)))))
                    })
                
                if abs(put_delta) > 0.01:
                    options_data.append({
                        'type': 'put',
                        'strike': strike,
                        'expiry': expiry,
                        'days_to_expiry': days_to_expiry,
                        'time_to_expiry': time_to_expiry,
                        'delta': put_delta,
                        'implied_vol': volatility,
                        'moneyness': strike / current_price,
                        'volume': max(1, int(80 * np.exp(-2 * abs(np.log(current_price/strike)))))
                    })
        
        return pd.DataFrame(options_data)
    
    def analyze_delta_patterns(self, options_df, current_price):
        """Analyze key delta patterns and relationships"""
        if options_df.empty:
            return None
        
        calls = options_df[options_df['type'] == 'call']
        puts = options_df[options_df['type'] == 'put']
        
        # Delta categorization
        deep_itm_calls = calls[calls['delta'] > 0.8]
        atm_calls = calls[(calls['delta'] >= 0.4) & (calls['delta'] <= 0.6)]
        otm_calls = calls[calls['delta'] < 0.2]
        
        deep_itm_puts = puts[puts['delta'] < -0.8]
        atm_puts = puts[(puts['delta'] >= -0.6) & (puts['delta'] <= -0.4)]
        otm_puts = puts[puts['delta'] > -0.2]
        
        # Delta-moneyness relationship
        call_delta_moneyness = calls.groupby('moneyness')['delta'].mean()
        put_delta_moneyness = puts.groupby('moneyness')['delta'].mean()
        
        # Time decay effect on delta
        delta_by_expiry = options_df.groupby(['expiry', 'type'])['delta'].mean().unstack()
        
        analysis = {
            'total_options': len(options_df),
            'call_count': len(calls),
            'put_count': len(puts),
            
            # Delta distribution
            'avg_call_delta': calls['delta'].mean(),
            'avg_put_delta': puts['delta'].mean(),
            'delta_std_calls': calls['delta'].std(),
            'delta_std_puts': puts['delta'].std(),
            
            # Delta categories
            'deep_itm_calls': len(deep_itm_calls),
            'atm_calls': len(atm_calls),
            'otm_calls': len(otm_calls),
            'deep_itm_puts': len(deep_itm_puts),
            'atm_puts': len(atm_puts),
            'otm_puts': len(otm_puts),
            
            # Key insights
            'highest_call_delta': calls['delta'].max(),
            'lowest_put_delta': puts['delta'].min(),
            'delta_range': options_df['delta'].max() - options_df['delta'].min(),
            
            # Volume-weighted delta
            'volume_weighted_call_delta': (calls['delta'] * calls['volume']).sum() / calls['volume'].sum() if calls['volume'].sum() > 0 else 0,
            'volume_weighted_put_delta': (puts['delta'] * puts['volume']).sum() / puts['volume'].sum() if puts['volume'].sum() > 0 else 0,
        }
        
        return analysis
    
    def plot_delta_analysis(self, options_df, current_price, symbol):
        """Create focused delta visualizations"""
        if options_df.empty:
            return
        
        calls = options_df[options_df['type'] == 'call']
        puts = options_df[options_df['type'] == 'put']
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle(f'Delta Analysis for {symbol} (Current: ${current_price:.2f})', fontsize=16)
        
        # 1. Delta vs Strike Price
        axes[0, 0].scatter(calls['strike'], calls['delta'], alpha=0.7, color='green', label='Calls', s=60)
        axes[0, 0].scatter(puts['strike'], puts['delta'], alpha=0.7, color='red', label='Puts', s=60)
        axes[0, 0].axvline(current_price, color='black', linestyle='--', linewidth=2, label=f'Current Price')
        axes[0, 0].axhline(0, color='gray', linestyle='-', alpha=0.3)
        axes[0, 0].axhline(0.5, color='blue', linestyle=':', alpha=0.5, label='Delta = 0.5')
        axes[0, 0].axhline(-0.5, color='blue', linestyle=':', alpha=0.5)
        axes[0, 0].set_xlabel('Strike Price')
        axes[0, 0].set_ylabel('Delta')
        axes[0, 0].set_title('Delta vs Strike Price')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # 2. Delta Distribution
        axes[0, 1].hist(calls['delta'], bins=15, alpha=0.7, label='Calls', color='green', density=True)
        axes[0, 1].hist(puts['delta'], bins=15, alpha=0.7, label='Puts', color='red', density=True)
        axes[0, 1].axvline(calls['delta'].mean(), color='green', linestyle='--', 
                          label=f'Call Avg: {calls["delta"].mean():.3f}')
        axes[0, 1].axvline(puts['delta'].mean(), color='red', linestyle='--',
                          label=f'Put Avg: {puts["delta"].mean():.3f}')
        axes[0, 1].set_xlabel('Delta')
        axes[0, 1].set_ylabel('Density')
        axes[0, 1].set_title('Delta Distribution')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # 3. Delta vs Moneyness
        axes[1, 0].scatter(calls['moneyness'], calls['delta'], alpha=0.7, color='green', label='Calls')
        axes[1, 0].scatter(puts['moneyness'], puts['delta'], alpha=0.7, color='red', label='Puts')
        axes[1, 0].axvline(1.0, color='black', linestyle='--', alpha=0.5, label='At-the-money')
        axes[1, 0].set_xlabel('Moneyness (S/K for calls, K/S for puts)')
        axes[1, 0].set_ylabel('Delta')
        axes[1, 0].set_title('Delta vs Moneyness')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # 4. Delta vs Time to Expiration
        axes[1, 1].scatter(calls['days_to_expiry'], calls['delta'], alpha=0.7, color='green', label='Calls')
        axes[1, 1].scatter(puts['days_to_expiry'], puts['delta'], alpha=0.7, color='red', label='Puts')
        axes[1, 1].set_xlabel('Days to Expiry')
        axes[1, 1].set_ylabel('Delta')
        axes[1, 1].set_title('Delta vs Time to Expiration')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def delta_scenario_analysis(self, options_df, current_price, price_scenarios=None):
        """Analyze how delta changes with different underlying prices"""
        if options_df.empty:
            return None
        
        if price_scenarios is None:
            price_scenarios = [
                current_price * 0.90,  # -10%
                current_price * 0.95,  # -5%
                current_price * 0.98,  # -2%
                current_price,         # Current
                current_price * 1.02,  # +2%
                current_price * 1.05,  # +5%
                current_price * 1.10   # +10%
            ]
        
        scenario_results = []
        
        for scenario_price in price_scenarios:
            change_pct = (scenario_price - current_price) / current_price * 100
            
            call_deltas = []
            put_deltas = []
            
            for _, row in options_df.iterrows():
                new_delta = self.black_scholes_delta(
                    scenario_price, row['strike'], row['time_to_expiry'],
                    self.risk_free_rate, row['implied_vol'], row['type']
                )
                
                if row['type'] == 'call':
                    call_deltas.append(new_delta)
                else:
                    put_deltas.append(new_delta)
            
            scenario_results.append({
                'price': scenario_price,
                'change_pct': change_pct,
                'avg_call_delta': np.mean(call_deltas) if call_deltas else 0,
                'avg_put_delta': np.mean(put_deltas) if put_deltas else 0,
                'max_call_delta': max(call_deltas) if call_deltas else 0,
                'min_put_delta': min(put_deltas) if put_deltas else 0
            })
        
        return pd.DataFrame(scenario_results)
    
    def run_delta_analysis(self, symbol, price_range=20):
        """Main delta analysis function"""
        print(f"🎯 DELTA-FOCUSED ANALYSIS FOR {symbol}")
        print("=" * 50)
        
        # Get current price
        current_price = self.get_current_price(symbol)
        if current_price is None:
            print("❌ Unable to fetch current price")
            return None
        
        print(f"Current Price: ${current_price:.2f}")
        
        # Generate options data
        options_df = self.generate_delta_focused_options(symbol, current_price, price_range)
        
        if options_df.empty:
            print("❌ No options data generated")
            return None
        
        print(f"Generated {len(options_df)} options for delta analysis")
        
        # Analyze delta patterns
        analysis = self.analyze_delta_patterns(options_df, current_price)
        
        # Display delta insights
        print(f"\n📊 DELTA INSIGHTS")
        print(f"-" * 30)
        print(f"Average Call Delta: {analysis['avg_call_delta']:.4f}")
        print(f"Average Put Delta: {analysis['avg_put_delta']:.4f}")
        print(f"Highest Call Delta: {analysis['highest_call_delta']:.4f}")
        print(f"Lowest Put Delta: {analysis['lowest_put_delta']:.4f}")
        print(f"Delta Range: {analysis['delta_range']:.4f}")
        
        print(f"\n📈 DELTA CATEGORIES")
        print(f"-" * 30)
        print(f"Deep ITM Calls (Δ>0.8): {analysis['deep_itm_calls']}")
        print(f"ATM Calls (0.4<Δ<0.6): {analysis['atm_calls']}")
        print(f"OTM Calls (Δ<0.2): {analysis['otm_calls']}")
        print(f"Deep ITM Puts (Δ<-0.8): {analysis['deep_itm_puts']}")
        print(f"ATM Puts (-0.6<Δ<-0.4): {analysis['atm_puts']}")
        print(f"OTM Puts (Δ>-0.2): {analysis['otm_puts']}")
        
        print(f"\n💰 VOLUME-WEIGHTED DELTAS")
        print(f"-" * 30)
        print(f"Volume-Weighted Call Delta: {analysis['volume_weighted_call_delta']:.4f}")
        print(f"Volume-Weighted Put Delta: {analysis['volume_weighted_put_delta']:.4f}")
        
        # Scenario analysis
        scenarios = self.delta_scenario_analysis(options_df, current_price)
        if scenarios is not None:
            print(f"\n🎢 DELTA SCENARIOS")
            print(f"-" * 30)
            for _, row in scenarios.iterrows():
                print(f"${row['price']:.2f} ({row['change_pct']:+.1f}%): "
                      f"Call Δ={row['avg_call_delta']:.3f}, Put Δ={row['avg_put_delta']:.3f}")
        
        # Show top delta options
        calls = options_df[options_df['type'] == 'call']
        puts = options_df[options_df['type'] == 'put']
        
        print(f"\n🏆 TOP DELTA OPTIONS")
        print(f"-" * 30)
        
        print("\nHighest Call Deltas:")
        top_calls = calls.nlargest(5, 'delta')[['strike', 'expiry', 'delta', 'moneyness']]
        print(top_calls.round(4).to_string(index=False))
        
        print("\nMost Negative Put Deltas:")
        top_puts = puts.nsmallest(5, 'delta')[['strike', 'expiry', 'delta', 'moneyness']]
        print(top_puts.round(4).to_string(index=False))
        
        # Create visualizations
        self.plot_delta_analysis(options_df, current_price, symbol)
        
        return options_df, analysis


# Test and demo functions
def test_delta_analyzer():
    """Test the delta analyzer"""
    if not ALPHA_VANTAGE_API_KEY:
        print("❌ No API key found. Please set ALPHA_VANTAGE_API_KEY in your .env file")
        return False
    
    analyzer = DeltaFocusedAnalyzer(ALPHA_VANTAGE_API_KEY)
    price = analyzer.get_current_price('AAPL')
    
    if price:
        print(f"✅ Delta analyzer ready - AAPL: ${price:.2f}")
        return True
    else:
        print("❌ Unable to connect to API")
        return False

def run_delta_demo(symbol='AAPL'):
    """Run a complete delta analysis demo"""
    if not test_delta_analyzer():
        return
    
    analyzer = DeltaFocusedAnalyzer(ALPHA_VANTAGE_API_KEY)
    return analyzer.run_delta_analysis(symbol)

# Ready to use!
print("🎯 Delta-Focused Options Analyzer Ready!")
print("\nKey Functions:")
print("- test_delta_analyzer(): Test API connection")
print("- run_delta_demo('AAPL'): Run complete delta analysis")
print("\nFocus Areas:")
print("- Delta calculations using Black-Scholes")
print("- Delta patterns and relationships")
print("- Delta categorization (ITM/ATM/OTM)")
print("- Delta scenario analysis")
print("- Delta visualizations")

if __name__ == "__main__":
    # Run demo
    run_delta_demo()

🎯 Delta-Focused Options Analyzer Ready!

Key Functions:
- test_delta_analyzer(): Test API connection
- run_delta_demo('AAPL'): Run complete delta analysis

Focus Areas:
- Delta calculations using Black-Scholes
- Delta patterns and relationships
- Delta categorization (ITM/ATM/OTM)
- Delta scenario analysis
- Delta visualizations
Request error: Expecting value: line 1 column 1 (char 0)
Request error: Expecting value: line 1 column 1 (char 0)
❌ Unable to connect to API
